In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/refs/heads/master/titanic.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# impute missing -> OneHot Encode -> Scaling -> feature selection -> train model

In [5]:
df.drop(columns = ['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace = True)

In [6]:
# STEP 0  -> Train test split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = ['Survived']), df['Survived'], test_size = 0.2, random_state = 42)

In [7]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [8]:
# STEP 1 -> Imputing Missing Values
tnf1 = ColumnTransformer(transformers=[
    ('impute_age', SimpleImputer(), [2]),
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6])
], remainder='passthrough')

In [9]:
# STEP 2  -> OneHot Encoding

tnf2 = ColumnTransformer(transformers=[
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1, 6])
], remainder='drop')

In [10]:
# STEP 3  -> Scaling

# (0, 10) bcz after OneHot Encoding, we have 10 columns
tnf3 = ColumnTransformer(transformers=[
    ('scale', MinMaxScaler(), slice(0, 10))
], remainder='passthrough')

In [11]:
# STEP 4  -> Feature Selection

tnf4 = SelectKBest(score_func=chi2, k=8)

In [12]:
# STEP 5  -> Train Model

tnf5 = DecisionTreeClassifier()

# Creating Pipeline

In [13]:
pipe = Pipeline([
    ('tnf1', tnf1),
    ('tnf2', tnf2),
    ('tnf3', tnf3),
    ('tnf4', tnf4),
    ('tnf5', tnf5)
])

### Pipeline vs make_pipeline


*   `Pipeline` requires naming of steps while `make_pipeline` does not.
*    similar is the case with `ColumnTranformer` and `make_column_transformer`





In [20]:
# Alternative Syntax
pipe = make_pipeline(tnf1, tnf2, tnf3, tnf4, tnf5)

* if we are ***not training our model in the pipeline***, then we have to use `fit_transform`. for example, suppose did only imputing, encoding and scaling, then we shall use `fit_transform`.
* if we are ***training the model in the pipeline*** like we did in `STEP 5`, we have to use `fit` only.

In [14]:
# training the pipe
pipe.fit(X_train, y_train)

Pipeline(steps=[('tnf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('tnf2',
                 ColumnTransformer(transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('tnf3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('tnf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7f01d53e4680>)),
                ('tnf5', DecisionTreeClassifier())])

In [25]:
# Predicting
y_pred = pipe.predict(X_test)

In [26]:
accuracy_score(y_test, y_pred)

0.6312849162011173

## **Exploring the Pipeline**

In [15]:
pipe.named_steps

{'tnf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'tnf2': ColumnTransformer(transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'tnf3': ColumnTransformer(remainder='passthrough',
                   transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'tnf4': SelectKBest(k=8, score_func=<function chi2 at 0x7f01d53e4680>),
 'tnf5': DecisionTreeClassifier()}

In [17]:
pipe.named_steps['tnf1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [19]:
pipe.named_steps['tnf1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder',
  FunctionTransformer(accept_sparse=True, check_inverse=False,
                      feature_names_out='one-to-one'),
  [0, 1, 3, 4, 5])]

In [20]:
pipe.named_steps['tnf1'].transformers_[0][1]

SimpleImputer()

In [22]:
# getting mean of first impute

pipe.named_steps['tnf1'].transformers_[0][1].statistics_

array([29.49884615])

In [23]:
# most frequently embarked
pipe.named_steps['tnf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

### **Cross Validation using PipeLine**

In [27]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

np.float64(0.6433270954397715)

### **GridSearch** **using** **Pipeline**

In [29]:
params = {
    'tnf5__max_depth':[1, 2, 3, 4, 5, None]
}

In [30]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tnf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('tnf2',
                                        ColumnTransformer(transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('tnf3',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('tnf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7f01d53e4680>)),
                                       ('tnf5', DecisionTreeClassifier())]),
             param_grid={'tnf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [31]:
grid.best_score_

np.float64(0.6447355461439969)

In [33]:
grid.best_params_

{'tnf5__max_depth': 5}

## **Exporting the Pipeline**

In [34]:
import pickle
pickle.dump(pipe, open('pipe.pkl', 'wb'))